In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from decorate import DecorateClassifier 
from data_generation_methods import *
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
np.warnings.filterwarnings('ignore')

TRAIN_PATH = 'data/abalone/abalone.data'
abalone = pd.read_csv(TRAIN_PATH, names = ["col" + str(i) for i in range(8)] + ['target']) 



In [2]:
def get_x_y():
    le = preprocessing.LabelEncoder() 
    y = abalone.target
    X = abalone.drop(columns='target', axis=1)
    return pd.get_dummies(X), le.fit_transform(y), le
X, y, le = get_x_y()

In [45]:
rf = RandomForestClassifier()
rf.fit(X,y)
y_rf = rf.predict_proba(X)

## test decorate method
def test_get():
    gen = DecorateDataGeneration()
    X_art = gen.gen_data(X, art_factor=1)
    y_art = gen.label_data(X_art, y_rf)
    return X_art, y_art
X_art, y_art = test_get()

In [4]:
# %%time
dec = DecorateClassifier(n_estimators=50, n_iter=100, art_factor=0.1)
dec.fit(X,y)

DecorateClassifier(base_estimator=<class 'sklearn.tree.tree.DecisionTreeClassifier'>,
          gen_artificial_method=<class 'DecorateDataGeneration', art_factor=0.1>,
          n_estimators=50, n_iter=100, random_seed=1.0)

In [ ]:
pred = dec.predict(np.random.rand(X.shape[0], X.shape[1]))
print("accuracy: {:.2f}".format(sum(pred == y)/len(X)))
print("ensemble size: ", len(dec.ensemble_))